In [1]:
import numpy as np
import re

In [156]:
fen_strings = ['rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
               'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq a8 0 1',
               'rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBNR w KQkq c6 0 2',
               'rnbqkbnr/pp1ppppp/8/2p5/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2']

In [160]:
def fen_to_input(fen):
    piece_translator = {'P' : [ 1, 0, 0, 0, 0, 0],
                        'R' : [ 0, 1, 0, 0, 0, 0],
                        'N' : [ 0, 0, 1, 0, 0, 0],
                        'B' : [ 0, 0, 0, 1, 0, 0],
                        'Q' : [ 0, 0, 0, 0, 1, 0],
                        'K' : [ 0, 0, 0, 0, 0, 1],
                        'p' : [-1, 0, 0, 0, 0, 0],
                        'r' : [ 0,-1, 0, 0, 0, 0],
                        'n' : [ 0, 0,-1, 0, 0, 0],
                        'b' : [ 0, 0, 0,-1, 0, 0],
                        'q' : [ 0, 0, 0, 0,-1, 0],
                        'k' : [ 0, 0, 0, 0, 0,-1],
                        ' ' : [ 0, 0, 0, 0, 0, 0]}

    result = np.zeros((450,), dtype=np.int8)
    
    board, to_move, castling_rights, en_passant, halfmove, fullmove = fen.split(' ')
    
    ## 
    board = board.replace('/', '')
    
    for i in range(1, 9):
        board = board.replace(str(i), ' ' * i)
        
    for i in range(len(board)):
        result[i:i+6] = piece_translator[board[i]]
    
    if to_move == 'w':
        result[384] = 1
    elif to_move == 'b':
        result[384] = -1
    
    if not en_passant == '-':
        row = 9 - int(en_passant[1]) - 1
        col = ord(en_passant[0]) - 96 - 1
        pos = col + (row * 8) - 1
        print(pos)
        result[385 + pos] = 1
        
    #if 'K' in castling_rights:
    #    result[386] = 1

    #if 'Q' in castling_rights:
    #    result[387] = 1

    #if 'k' in castling_rights:
    #    result[388] = -1

    #if 'q' in castling_rights:
    #    result[389] = -1
    
    

    return result

In [161]:
fen_to_input(fen_strings[1])

-1


array([ 0,  0,  0,  0,  0,  0,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,
        1,  0,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [141]:
386+64

450